In [38]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
%matplotlib qt

In [39]:
file = '../IQ_Cap/03-17-2021_150003'
Dfile = '../IQ_Cap/Drone-03-17-2021_153631'
NumSamp = 1024

samples = np.fromfile(file + '.iq', np.complex64)
drone_samp = np.fromfile(Dfile + '.iq', np.complex64)

In [40]:
params = dict()
with open(file + '.txt') as f:
    split = [a.strip('\n').split(': ') for a in f.readlines()]
    for i in split:
        params[i[0]] = int(i[1])    
    f.close()

In [41]:
Dparams = dict()
with open(Dfile + '.txt') as f:
    split = [a.strip('\n').split(': ') for a in f.readlines()]
    for i in split:
        Dparams[i[0]] = int(i[1]) if 'Drone' not in i[0] else i[1]
    f.close()

In [42]:
params

{'LO': 2440500000,
 'BW': 56000000,
 'Sample Rate': 40000000,
 'Buffer Size': 1024}

In [43]:
Dparams

{'LO': 2440500000,
 'BW': 56000000,
 'Sample Rate': 40000000,
 'Buffer Size': 1024,
 'Drone BW': '10Mhz',
 'Drone Center Freq': '2.4405'}

In [44]:
samp_range = np.arange(0,NumSamp) #+ params['Buffer Size']*3

In [45]:
len(samples)

1024000

In [35]:
def FFT(samples, buf, samprate, lo):
    fft = np.fft.fftshift(np.fft.fft(samples)) / buf
    freqLabels = np.fft.fftshift(np.fft.fftfreq(buf, 1/samprate)) + lo
    fft_mag_dB = 10*np.log10(np.abs(fft))
#     fft_mag = np.abs(fft)
    return fft_mag_dB, freqLabels

def MA(data):
    N = 10
    w = np.ones(N) / N
    ma = np.convolve(data, w, mode='same')
    return(ma)

def findSigPts(boolarray):
    a = boolarray^np.concatenate(([False],boolarray[:-1]))
    return(abs(a))

def findBWs(sigpts):
    freqs = np.zeros([len(sigpts)-1,2])
    freqs[:,0] = sigpts[1:]
    freqs[:,1] = sigpts[:-1]
    BWs = (sigpts[1:]-sigpts[:-1])/1e6
    return BWs, freqs

def findSigSec(boolarray):
    SigSections = []
    for i in range(1,len(boolarray)):
        if boolarray[i-1]:
            if i == 1:
                SigSections.append(1)
            if not boolarray[i]:
                SigSections.append(0)
        elif not boolarray[i-1]:
            if boolarray[i]:
                SigSections.append(1)

    return np.array(SigSections[:-1]).astype(np.bool)

In [36]:
freqData, freqLabels = FFT(samples[samp_range], NumSamp, params['Sample Rate'], params['LO'])
droneFreqData, droneFreqLabels = FFT(drone_samp[samp_range], NumSamp, Dparams['Sample Rate'], Dparams['LO'])

freq_ndb = 10**(droneFreqData/10)
freq_ma = MA(droneFreqData)
freq_ma_ndb = MA(freq_ndb)
ma_thresh = freq_ndb.mean()

BWs, freqRgs= findBWs(droneFreqLabels[findSigPts(freq_ma_ndb > ma_thresh)])
sigSec = findSigSec(freq_ma_ndb > ma_thresh)
sigBWs = BWs[sigSec][(BWs[sigSec] > 9) & (BWs[sigSec] < 11)]
drone_sig_range = freqRgs[sigSec][(BWs[sigSec] > 9) & (BWs[sigSec] < 11)]

In [37]:
timeData = dict()
timeData['I'] = np.real(samples[samp_range])
timeData['Q'] = np.imag(samples[samp_range])

droneTimeData = dict()
droneTimeData['I'] = np.real(drone_samp[samp_range])
droneTimeData['Q'] = np.imag(drone_samp[samp_range])

fig = plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.title('Nominal Time Domain')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
Nti, = plt.plot(timeData['I'], label='I')
Ntq, = plt.plot(timeData['Q'], label='Q')
plt.subplot(2,2,2)
plt.title('Drone Time Domain')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
Dti, = plt.plot(droneTimeData['I'], label='I')
Dtq, = plt.plot(droneTimeData['Q'], label='Q')
plt.subplot(2,2,3)
plt.title('Nominal Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Magnitude')
Nf, = plt.plot(freqLabels, freqData, label='Nominal')
sp = plt.subplot(2,2,4)
plt.title('Drone Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Magnitude')
Df, = plt.plot(droneFreqLabels, droneFreqData, label='Drone Present')
Mf, = plt.plot(droneFreqLabels[freq_ma_ndb > ma_thresh],freq_ma[freq_ma_ndb > ma_thresh])

fig.text(.3,.95,f"LO: {params['LO']} | Buffer: {params['Buffer Size']} | Sample Rate: {params['Sample Rate']}" , fontsize=12)
fig.text(.3,.92,f"RF Bandwidth: {params['BW']} | Drone Bandwidth: {Dparams['Drone BW']} | Drone Center Frequency: {Dparams['Drone Center Freq']}", fontsize=12)


text = fig.text(.85,.935,f"Samples: {samp_range[0]} to {params['Buffer Size']}", fontsize=10)
d_text = fig.text(.55,.045,f"Drone Signal Range: {drone_sig_range} GHz", fontsize=10)
# s_text = fig.text(.15,.025,f"Significant Points: {droneFreqLabels[findSigPts(freq_ma_ndb>ma_thresh)]}", fontsize=10)
b_text = fig.text(.55,.025,f"Bandwidths: {sigBWs}", fontsize=10)



class Index():
    ind = 0

    def next(self, event):
        self.ind += NumSamp
        i = self.ind % len(samples)//NumSamp
        
        Ntupi = np.real(samples[samp_range + params['Buffer Size']*i])
        Ntupq = np.imag(samples[samp_range + params['Buffer Size']*i])
        
        Dtupi = np.real(drone_samp[samp_range + params['Buffer Size']*i])
        Dtupq = np.imag(drone_samp[samp_range + params['Buffer Size']*i])
        
        freqData, freqLabels = FFT(samples[samp_range + params['Buffer Size']*i], NumSamp, params['Sample Rate'], params['LO'])
        droneFreqData, droneFreqLabels = FFT(drone_samp[samp_range + params['Buffer Size']*i], NumSamp, Dparams['Sample Rate'], Dparams['LO'])
        freq_ndb = 10**(droneFreqData/10)
        freq_ma = MA(droneFreqData)
        freq_ma_ndb = MA(freq_ndb)
        ma_thresh = freq_ndb.mean()
       
        BWs, freqRgs= findBWs(droneFreqLabels[findSigPts(freq_ma_ndb > ma_thresh)])
        sigSec = findSigSec(freq_ma_ndb > ma_thresh)
        sigBWs = BWs[sigSec][(BWs[sigSec] > 9) & (BWs[sigSec] < 11)]
        drone_sig_range = freqRgs[sigSec][(BWs[sigSec] > 9) & (BWs[sigSec] < 11)]  
        
        Nti.set_ydata(Ntupi)
        Ntq.set_ydata(Ntupq)
        
        Dti.set_ydata(Ntupi)
        Dtq.set_ydata(Ntupq)

        Nf.set_xdata(freqLabels)
        Nf.set_ydata(freqData)
        Df.set_xdata(droneFreqLabels)
        Df.set_ydata(droneFreqData)
        Mf.set_xdata(droneFreqLabels[freq_ma_ndb > ma_thresh])
        Mf.set_ydata(freq_ma[freq_ma_ndb > ma_thresh])
        
        
        
        
        #print("Next!")
        
        text.set_text(f"Samples: {samp_range[0] + params['Buffer Size']*i} to {samp_range[-1] + params['Buffer Size']*i}")
        d_text.set_text(f"Drone Signal Range: {drone_sig_range} GHz")
#         s_text.set_text(f"Significant Points: {droneFreqLabels[findSigPts(freq_ma_ndb>ma_thresh)]}")
        b_text.set_text(f"Bandwidths: {sigBWs}")


        plt.draw()

    def prev(self, event):
        self.ind -= NumSamp
        i = self.ind % len(samples)//NumSamp
        
        Ntupi = np.real(samples[samp_range + params['Buffer Size']*i])
        Ntupq = np.imag(samples[samp_range + params['Buffer Size']*i])
        
        Dtupi = np.real(drone_samp[samp_range + params['Buffer Size']*i])
        Dtupq = np.imag(drone_samp[samp_range + params['Buffer Size']*i])
        
        freqData, freqLabels = FFT(samples[samp_range + params['Buffer Size']*i], NumSamp, params['Sample Rate'], params['LO'])
        droneFreqData, droneFreqLabels = FFT(drone_samp[samp_range + params['Buffer Size']*i], NumSamp, Dparams['Sample Rate'], Dparams['LO'])
        freq_ndb = 10**(droneFreqData/10)
        freq_ma = MA(droneFreqData)
        freq_ma_ndb = MA(freq_ndb)
        ma_thresh = freq_ndb.mean()

        BWs, freqRgs= findBWs(droneFreqLabels[findSigPts(freq_ma_ndb > ma_thresh)])
        sigSec = findSigSec(freq_ma_ndb > ma_thresh)
        sigBWs = BWs[sigSec][(BWs[sigSec] > 9) & (BWs[sigSec] < 11)]
        drone_sig_range = freqRgs[sigSec][(BWs[sigSec] > 9) & (BWs[sigSec] < 11)]  

        
        Nti.set_ydata(Ntupi)
        Ntq.set_ydata(Ntupq)
        
        Dti.set_ydata(Ntupi)
        Dtq.set_ydata(Ntupq)

        Nf.set_xdata(freqLabels)
        Nf.set_ydata(freqData)
        Df.set_xdata(droneFreqLabels)
        Df.set_ydata(droneFreqData)
        Mf.set_xdata(droneFreqLabels[freq_ma_ndb > ma_thresh])
        Mf.set_ydata(freq_ma[freq_ma_ndb > ma_thresh])
        
        
        #print("Prev!")
        
        text.set_text(f"Samples: {samp_range[0] + params['Buffer Size']*i} to {samp_range[-1] + params['Buffer Size']*i}")
        d_text.set_text(f"Drone Signal Range: {drone_sig_range} GHz")
#         s_text.set_text(f"Significant Points: {droneFreqLabels[findSigPts(freq_ma_ndb>ma_thresh)]}")
        b_text.set_text(f"Bandwidths: {sigBWs}")

        
        plt.draw()

callback = Index()
axprev = plt.axes([0.9, 0.43, 0.1, 0.075])
axnext = plt.axes([0.9, 0.53, 0.1, 0.075])
bnext = Button(axnext, 'Next')
bnext.on_clicked(callback.next)
bprev = Button(axprev, 'Previous')
bprev.on_clicked(callback.prev)

0

In [18]:
mvAvg = []
for i in droneFreqData:
    i

plt.figure()
plt.plot(droneFreqLabels, droneFreqData, label='Drone Present')
plt.show()